# Importing Libraries

In [4]:
import pandas as pd
pd.set_option('display.width', 500)

# Data Importing

I will be supplementing the original Kaggle Dataset with another dataset containing the play-by-play for 2019.

In [10]:
raw_data = pd.read_csv("Data/NFL Play by Play 2009-2018 (v5).csv.zip", encoding='utf-8')
raw_data

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa3 in position 11: invalid start byte

In [6]:
raw_data_2019 = pd.read_csv("Data/reg_pbp_2019.csv.zip")
raw_data_2019

FileNotFoundError: [Errno 2] No such file or directory: 'Data/reg_pbp_2019.csv.zip'

In [ ]:
raw_data = pd.concat([raw_data,raw_data_2019])
raw_data

In [ ]:
raw_data.head(20)

In [ ]:
raw_data.info(verbose = True)

In [ ]:
for col in raw_data:
    print(raw_data[col].value_counts())

In [ ]:
raw_data['home_team'].nunique()

There are currently 32 teams within the NFL but the reason that there are 35 teams listed is due to two teams relocating and one team changing their initials. For ease, I will combine the teams 'STL/LA', 'SD/LAC' , and change 'JAC' to JAX'.

In [ ]:
raw_data = raw_data.replace('JAC','JAX')
raw_data['home_team'].value_counts()

In [ ]:
raw_data = raw_data.replace('SD','SD/LAC')
raw_data = raw_data.replace('LAC','SD/LAC')
raw_data = raw_data.replace('LA','STL/LA')
raw_data = raw_data.replace('STL','STL/LA')
raw_data['home_team'].nunique()

In [ ]:
raw_data['home_team'].value_counts()

Next, I want to filter out the plays that were not runs or passes. There are number of other plays that can occur within football such as field goals, as well as plays that don't occur as a result of penalties. I wanted to see why there were two different designations for 'no play' and null plays.

In [ ]:
raw_data['play_type'].value_counts(dropna = False)

In [ ]:
pd.options.display.max_colwidth = 100

null_play = raw_data[raw_data['play_type'].isna()]
null_play['desc'].head(50)

Plays that are designated null appear to be mostly based on period changes such as the end of a quarter or the end of the game.

In [ ]:
null_play.head(10)

In [ ]:
not_play = raw_data[raw_data['play_type'] == 'no_play']
not_play['desc'].head(50)

Plays deemed as no plays were mostly due to timeouts called and plays that were called off due to penalties. I can now proceed with selecting plays that occurred without penalties and without fault.

In [ ]:
run_pass_plays = raw_data.loc[(raw_data.play_type == 'run') | (raw_data.play_type == 'pass')]
run_pass_plays

In [ ]:
run_pass_plays['play_type'].value_counts()

In [ ]:
for col in run_pass_plays:
    print(run_pass_plays[col].value_counts(dropna = False))

In [ ]:
run_pass_plays.info(verbose = True, null_counts = True)

In [ ]:
pass_plays = run_pass_plays[run_pass_plays['play_type'] == 'pass']
pass_plays['pass_length'].value_counts()

Next, I need to subset the relevant columns for analysis. Even though the dataset contains a vast number of columns for interesting information such as expected points added and win probability added, I will only be using columns that pertain to details of the play and win probability.

In [ ]:
merge_part_1 = run_pass_plays[run_pass_plays.columns[0:57]]
merge_part_2 = run_pass_plays[run_pass_plays.columns[87:89]]
merge_part_2

In [ ]:
concat_df = pd.concat([merge_part_1, merge_part_2], axis = 1)
concat_df

In [ ]:
concat_df.info()

I can remove non-relevant columns that still remain after this such as the ones that pertain to field goals and kicks.

In [ ]:
concat_df = concat_df.drop(['field_goal_result','kick_distance', 'extra_point_result','two_point_conv_result'], axis = 1)
concat_df.info()

One peculiar feature about this dataset is that it contains separate columns for both home/away and possessing/defending teams. The posteam column pertains to whether the team currently possessing the ball is playing home or away. For now, we will remove the columns that are redundant.

In [ ]:
for col in concat_df:
    print(concat_df[col].value_counts())

In [ ]:
non_home_away_df = concat_df

In [ ]:
non_home_away_df = non_home_away_df.drop (['home_team','away_team','home_timeouts_remaining',
                                          'away_timeouts_remaining', 'total_home_score',
                                          'total_away_score'], axis = 1)
non_home_away_df

In [ ]:
non_home_away_df.info()

Next, I wanted to look at some of the columns as to why they contained null values.

In [ ]:
non_home_away_df[non_home_away_df.down.isna()].desc.head(50)

This is an interesting situation because two point conversions don't necessarily count as a down. They are done after a team scores a touchdown and a team has only one chance at executing the play correctly (if there are no penalties or other circumstances). I could choose to either ignore two-point conversions or evaluate them to as though they were similar to a 4th down play. Two point conversions are commonplace when a team is down by multiple scores and trying to catch up.

Next, I wanted to see why certain plays had missing values with regards to the time remaining.

In [ ]:
non_home_away_df[non_home_away_df.half_seconds_remaining.isna()].desc

In [ ]:
non_home_away_df[non_home_away_df.game_seconds_remaining.isna()].desc

For some peculiar reason, the missing data was a result of the time left in the quarter being at some multiple of 100 seconds (1:40,3:20,5:00)

In [ ]:
non_home_away_df[non_home_away_df.wp.isna()].desc

In [ ]:
non_home_away_df.info()

There are some columns that pertain to whether a timeout was called on a play and which team called it. Since we already have the columns regarding the timeouts remaining for each team, these columns are not needed.

In [ ]:
non_home_away_df = non_home_away_df.drop(['timeout_team', 'timeout'], axis = 1)
non_home_away_df

In [ ]:
non_home_away_df.info()

Now it's time to deal with the columns with specific details about each play, for passes and runs. First, I'll deal with run plays.

In [ ]:
print(non_home_away_df.run_gap.value_counts(dropna = False))

print(non_home_away_df.run_location.value_counts(dropna = False))

In [ ]:
run_plays_only = non_home_away_df[non_home_away_df.play_type == 'run']
run_plays_only

In [ ]:
print(run_plays_only.run_gap.value_counts(dropna = False))

print(run_plays_only.run_location.value_counts(dropna = False))

In [ ]:
run_plays_only[run_plays_only.run_gap.isna()].desc.head(25)

These plays won't have run gaps since they were runs up the middle. A run gap refers to the gap of space made by multiple players on the offensive line. As these were runs up the middle, there wasn't necessarily a "gap" created so these will have their run gap changed from null to be "middle"

In [ ]:
run_plays_only[run_plays_only.run_location.isna()].desc.head(50)

These mostly refer to plays in which the quarterback fumbles the ball and tries to advance the ball. These are not designed plays. Some instances involve a quarterback kneeling the ball as well. A quarterback kneels in order to expend as much time as possible off the clock and typically come at the end of a game. I will go ahead and remove these plays entirely.